# Remote actions and variables

Changing the state of remote resources


---
- Website: https://discovery.gitlabpages.inria.fr/enoslib/index.html
- Instant chat: https://framateam.org/enoslib
---




**Prerequisites**:
- A Grid'5000 account
- A working EnOSlib environment and Jupyter (not included in EnOSlib dependencies, but `pip install jupyterlab` will install it)




In [1]:
import enoslib as en

# get some logging info
import logging
logging.basicConfig(level=logging.INFO)

## Setup on Grid'5000

### Describing the resources

For the purpose of the tutorial we'll reserve 2 nodes in the production environment.

First we build a configuration object describing the wanted resources: `machines` and `networks`.

In [2]:
network = en.G5kNetworkConf(id="n1", type="prod", roles=["my_network"], site="rennes")

conf = (
    en.G5kConf.from_settings(job_type="allow_classic_ssh", job_name="rsd-01")
    .add_network_conf(network)
    .add_machine(
        roles=["control"], cluster="parasilo", nodes=1, primary_network=network
    )
    .add_machine(
        roles=["compute"],
        cluster="parasilo",
        nodes=1,
        primary_network=network,
    )
    .finalize()
)
conf

Conf@0x7f820b795b90
{
    "dhcp": true,
    "force_deploy": false,
    "env_name": "debian10-x64-nfs",
    "job_name": "rsd-01",
    "job_type": "allow_classic_ssh",
    "key": "/home/msimonin/.ssh/id_rsa.pub",
    "queue": "default",
    "walltime": "02:00:00",
    "resources": {
        "machines": [
            {
                "roles": [
                    "control"
                ],
                "primary_network": "n1",
                "secondary_networks": [],
                "cluster": "parasilo",
                "nodes": 1
            },
            {
                "roles": [
                    "compute"
                ],
                "primary_network": "n1",
                "secondary_networks": [],
                "cluster": "parasilo",
                "nodes": 1
            }
        ],
        "networks": [
            {
                "id": "n1",
                "type": "prod",
                "roles": [
                    "my_network"
                ],
                "site": "rennes"
            }
        ]
    }
}

### Reserving the resources

We can pass the `Configuration` object to the `G5k` provider. The provider is responsible to turn the abstract resource description into a et concrete resources.

In [3]:
provider = en.G5k(conf)
roles, networks = provider.init()

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Submitting {'name': 'rsd-01', 'types': ['allow_classic_ssh'], 'resources': "{cluster='parasilo'}/nodes=1+{cluster='parasilo'}/nodes=1,walltime=02:00:00", 'command': 'sleep 31536000', 'queue': 'default'} on rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1816396 on rennes [2021-08-27 16:52:56]
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1816396 on rennes [2021-08-27 16:52:56]
INFO:enoslib.infra.enos_g5k.g5k_api_utils:All jobs are Runni

Inspecting the ressources we own for the experiment's lifetime:

- roles: this is somehow a dictionnary whose keys are the role names and the associated values are the corresponding list of hosts
- networks: similar to roles bu£t for networks

In [4]:
roles

{'control': [Host(address='parasilo-12.rennes.grid5000.fr', alias='parasilo-12.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())], 'compute': [Host(address='parasilo-20.rennes.grid5000.fr', alias='parasilo-20.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())]}

In [5]:
# list of host on a given role
roles["control"]

[Host(address='parasilo-12.rennes.grid5000.fr', alias='parasilo-12.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())]

In [6]:
# a single host
roles["control"][0]

Host(address='parasilo-12.rennes.grid5000.fr', alias='parasilo-12.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())

In [7]:
networks

{'my_network': [<enoslib.infra.enos_g5k.objects.G5kEnosProd4Network object at 0x7f820b7891d0>, <enoslib.infra.enos_g5k.objects.G5kEnosProd6Network object at 0x7f8234303450>]}

`provider.init` is idempotent. In the Grid'5000 case, you can call it several time in a row. The same reservation will reloaded and the roles and networks will be the same.

In [8]:
roles, networks = provider.init()
roles

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading 1816396 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1816396 on rennes [2021-08-27 16:52:56]
INFO:enoslib.infra.enos_g5k.g5k_api_utils:All jobs are Running !


{'control': [Host(address='parasilo-12.rennes.grid5000.fr', alias='parasilo-12.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())], 'compute': [Host(address='parasilo-20.rennes.grid5000.fr', alias='parasilo-20.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())]}

In [10]:
# sync some more information in the host data structure (for illustration purpose here)
roles = en.sync_info(roles, networks)


PLAY [all] ************************************************************************************************************************************************

TASK [hostname] *******************************************************************************************************************************************
 [started TASK: hostname on parasilo-12.rennes.grid5000.fr]
 [started TASK: hostname on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]

PLAY [Gather facts for all hosts] *************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [parasilo-20.rennes.grid5000.fr]
ok: [parasilo-12.rennes.grid5000.fr]

TASK [setup] ********************************************************************************

## Acting on remote nodes

### run a command, filter results

In [11]:
results = en.run_command("whoami", roles=roles)
results


PLAY [all] ************************************************************************************************************************************************

TASK [whoami] *********************************************************************************************************************************************
 [started TASK: whoami on parasilo-12.rennes.grid5000.fr]
 [started TASK: whoami on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]


CommandResult(host='parasilo-12.rennes.grid5000.fr', task='whoami', status='OK', payload={'cmd': 'whoami', 'stdout': 'root', 'stderr': '', 'rc': 0, 'start': '2021-08-27 16:53:45.941282', 'end': '2021-08-27 16:53:45.950134', 'delta': '0:00:00.008852', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'whoami', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['root'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-20.rennes.grid5000.fr', task='whoami', status='OK', payload={'cmd': 'whoami', 'stdout': 'root', 'stderr': '', 'rc': 0, 'start': '2021-08-27 16:53:45.941442', 'end': '2021-08-27 16:53:45.950115', 'delta': '0:00:00.008673', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'whoami', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['root'], 'stderr_lines': [], '_ansible_no_log': False})

In [12]:
one_result = results.filter(host=roles["control"][0].alias)[0]
one_result

CommandResult(host='parasilo-12.rennes.grid5000.fr', task='whoami', status='OK', payload={'cmd': 'whoami', 'stdout': 'root', 'stderr': '', 'rc': 0, 'start': '2021-08-27 16:53:45.941282', 'end': '2021-08-27 16:53:45.950134', 'delta': '0:00:00.008852', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'whoami', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['root'], 'stderr_lines': [], '_ansible_no_log': False})

In [13]:
one_result.payload["stdout"]

'root'

There are some specific shortcuts when the remote actions is a remote (shell) command: `.stdout`, `.stderr`, `.rc`

In [14]:
print(f"stdout = {one_result.stdout}\n", f"stderr={one_result.stderr}\n", f"return code = {one_result.rc}")

stdout = root
 stderr=
 return code = 0


By default the user is `root` (this is common to all EnOSlib's provider).
If you want to run command as your regular Grid'5000 user you can tell the command to `sudo` back to your regular user using `run_as` (the SSH login is still `root` though)

In [16]:
my_g5k_login = en.g5k_api_utils.get_api_username()
results = en.run_command("whoami", roles=roles, run_as=my_g5k_login)
results


PLAY [all] ************************************************************************************************************************************************

TASK [whoami] *********************************************************************************************************************************************
 [started TASK: whoami on parasilo-12.rennes.grid5000.fr]
 [started TASK: whoami on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]


CommandResult(host='parasilo-12.rennes.grid5000.fr', task='whoami', status='OK', payload={'cmd': 'whoami', 'stdout': 'msimonin', 'stderr': '', 'rc': 0, 'start': '2021-08-27 16:57:08.811766', 'end': '2021-08-27 16:57:08.814615', 'delta': '0:00:00.002849', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'whoami', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['msimonin'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-20.rennes.grid5000.fr', task='whoami', status='OK', payload={'cmd': 'whoami', 'stdout': 'msimonin', 'stderr': '', 'rc': 0, 'start': '2021-08-27 16:57:08.857367', 'end': '2021-08-27 16:57:08.860038', 'delta': '0:00:00.002671', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'whoami', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['msimonin'], 'stderr_lines': [], '_ansible_no_log': False})

### Filtering hosts on which the command is run

`run_command` acts on remote hosts. Those hosts can be given as a `Roles` type (output of `provider.init`) or as a list of `Host` or a single `Host`. 


In [17]:
# some roles
en.run_command("date", roles = roles)


PLAY [all] ************************************************************************************************************************************************

TASK [date] ***********************************************************************************************************************************************
 [started TASK: date on parasilo-12.rennes.grid5000.fr]
 [started TASK: date on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]


CommandResult(host='parasilo-12.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Fri 27 Aug 2021 04:57:15 PM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-27 16:57:15.154825', 'end': '2021-08-27 16:57:15.157409', 'delta': '0:00:00.002584', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Fri 27 Aug 2021 04:57:15 PM CEST'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-20.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Fri 27 Aug 2021 04:57:15 PM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-27 16:57:15.176568', 'end': '2021-08-27 16:57:15.179103', 'delta': '0:00:00.002535', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Fri 27 Aug 2021 04:57:15 PM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

In [18]:
# a list of hosts
en.run_command("date", roles = roles["control"])


PLAY [all] ************************************************************************************************************************************************

TASK [date] ***********************************************************************************************************************************************
 [started TASK: date on parasilo-12.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]


CommandResult(host='parasilo-12.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Fri 27 Aug 2021 04:57:16 PM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-27 16:57:16.359337', 'end': '2021-08-27 16:57:16.361706', 'delta': '0:00:00.002369', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Fri 27 Aug 2021 04:57:16 PM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

In [19]:
# a single host
en.run_command("date", roles=roles["control"][0])


PLAY [all] ************************************************************************************************************************************************

TASK [date] ***********************************************************************************************************************************************
 [started TASK: date on parasilo-12.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]


CommandResult(host='parasilo-12.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Fri 27 Aug 2021 04:57:17 PM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-27 16:57:17.339503', 'end': '2021-08-27 16:57:17.341813', 'delta': '0:00:00.002310', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Fri 27 Aug 2021 04:57:17 PM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

A `pattern_hosts` can also be supplied. The pattern can be a regexp but [other patterns are possible](
https://docs.ansible.com/ansible/latest/user_guide/intro_patterns.html#common-patterns)

In [20]:
# co* matches all hosts
en.run_command("date", roles=roles, pattern_hosts="co*")

# com* only matches host with `compute` tags
en.run_command("date", roles=roles, pattern_hosts="com*")


PLAY [co*] ************************************************************************************************************************************************

TASK [date] ***********************************************************************************************************************************************
 [started TASK: date on parasilo-12.rennes.grid5000.fr]
 [started TASK: date on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]

PLAY [com*] ***********************************************************************************************************************************************

TASK [date] ***********************************************************************************************************************************************
 [started TASK: date on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]


CommandResult(host='parasilo-20.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Fri 27 Aug 2021 04:57:23 PM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-27 16:57:23.102933', 'end': '2021-08-27 16:57:23.105307', 'delta': '0:00:00.002374', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Fri 27 Aug 2021 04:57:23 PM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

In [21]:
# you can forge some host yourself
# Here we run the command on the frontend: this should work if your SSH parameters are correct
en.run_command("date", roles=en.Host("rennes.grid5000.fr", user=en.g5k_api_utils.get_api_username()))


PLAY [all] ************************************************************************************************************************************************

TASK [date] ***********************************************************************************************************************************************
 [started TASK: date on rennes.grid5000.fr]
changed: [rennes.grid5000.fr]


CommandResult(host='rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Fri 27 Aug 2021 04:58:00 PM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-27 16:58:00.651786', 'end': '2021-08-27 16:58:00.689756', 'delta': '0:00:00.037970', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Fri 27 Aug 2021 04:58:00 PM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

### Remote actions

Tools like Ansible, Puppet, Chef, Terraform ... are shipped with a set of predefined remote actions to ease the administrator life.

Actions like copying file, adding some users, managing packages, making sure a line is absent from a configuration file, managing docker containers ... are first-class citizens actions and brings some nice garantees of correctness and idempotency.

There are 1000+ modules  available:
https://docs.ansible.com/ansible/2.9/modules/list_of_all_modules.html

---

EnOSlib wraps Ansible module and let you use them from Python (without writting any YAML file). You can call any module by using the `actions` context manager:

In the following we install docker (using g5k provided script) and a docker container. We also need to install the python docker binding on the remote machine so that Ansible can interact with the docker daemons on the remote machines. This block of actions is idempotent.


In [22]:
with en.actions(roles=roles) as a:
    # installing the docker daemon
    # prepending with a guard to make the command idempotent
    a.shell("which docker || /grid5000/code/bin/g5k-setup-docker")
    # install the python docker binding on the remote host
    # mandatory by the docker_container module
    a.pip(name="docker", state="present")
    # fire up a container (forward port 80 at the host level)
    a.docker_container(name="myserver", image="nginx", state="started", ports=["80:80"])
    # wait for the connection on the port 80 to be ready
    a.wait_for(port=80, state="started")
    # keep track of the result of each modules
    # not mandatory but nice :)
    results = a.results


PLAY [all] ************************************************************************************************************************************************

TASK [which docker || /grid5000/code/bin/g5k-setup-docker] ************************************************************************************************
 [started TASK: which docker || /grid5000/code/bin/g5k-setup-docker on parasilo-12.rennes.grid5000.fr]
 [started TASK: which docker || /grid5000/code/bin/g5k-setup-docker on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]

TASK [pip] ************************************************************************************************************************************************
 [started TASK: pip on parasilo-12.rennes.grid5000.fr]
 [started TASK: pip on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]

TASK [docker_container] ********************

[DEPRECATION WARNING]: The container_default_behavior option will change its default value from "compatibility" to "no_defaults" in community.docker 
2.0.0. To remove this warning, please specify an explicit value for it now. This feature will be removed from community.docker in version 2.0.0. 
Deprecation warnings can be disabled by setting deprecation_warnings=False in ansible.cfg.


changed: [parasilo-20.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]

TASK [wait_for] *******************************************************************************************************************************************
 [started TASK: wait_for on parasilo-12.rennes.grid5000.fr]
 [started TASK: wait_for on parasilo-20.rennes.grid5000.fr]
ok: [parasilo-12.rennes.grid5000.fr]
ok: [parasilo-20.rennes.grid5000.fr]


In [23]:
results.filter(task="docker_container")[0]

HostIp,HostPort
0.0.0.0,80
HostIp,HostPort
0.0.0.0,80


### Background actions

Sometime you need to fire a process on some remote machines that needs to survive the remote connection that started it. EnOSlib provides a `keyword` argument for this purpose and can be used when calling modules (when supported).

In [24]:
# synchronous execution, will wait until the end of the shell command
results = en.run_command("for i in $(seq 1 10); do sleep 1; echo toto; done", roles=roles)
results


PLAY [all] ************************************************************************************************************************************************

TASK [for i in $(seq 1 10); do sleep 1; echo toto; done] **************************************************************************************************
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-12.rennes.grid5000.fr]
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]


CommandResult(host='parasilo-12.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'cmd': 'for i in $(seq 1 10); do sleep 1; echo toto; done', 'stdout': 'toto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto', 'stderr': '', 'rc': 0, 'start': '2021-08-27 17:00:09.520458', 'end': '2021-08-27 17:00:19.541485', 'delta': '0:00:10.021027', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'for i in $(seq 1 10); do sleep 1; echo toto; done', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-20.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'cmd': 'for i in $(seq 1 10); do sleep 1; echo toto; done', 'stdout': 'toto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto', 'stderr': '', 'rc': 0, 'start': '2021-08-27 17:00:09.538001', 'end': '2021-08-27 17:00:19.560922', 'delta': '0:00:10.022921', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'for i in $(seq 1 10); do sleep 1; echo toto; done', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto'], 'stderr_lines': [], '_ansible_no_log': False})

In [25]:
# The remote command will be daemonize on the remote machines
results = en.run_command("for i in $(seq 1 10); do sleep 1; echo toto; done", roles=roles, background=True)
results


PLAY [all] ************************************************************************************************************************************************

TASK [for i in $(seq 1 10); do sleep 1; echo toto; done] **************************************************************************************************
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-12.rennes.grid5000.fr]
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]


AsyncCommandResult(host='parasilo-20.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'started': 1, 'finished': 0, 'ansible_job_id': '159355477779.1552', 'results_file': '/root/.ansible_async/159355477779.1552', 'changed': True, '_ansible_no_log': False})
AsyncCommandResult(host='parasilo-12.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'started': 1, 'finished': 0, 'ansible_job_id': '882274803934.27727', 'results_file': '/root/.ansible_async/882274803934.27727', 'changed': True, '_ansible_no_log': False})

In [29]:
# you can get back the status of the daemonized process by reading the remote results_file
h  = roles["control"][0]
result_file = results.filter(host=h.alias)[0].results_file
cat_result = en.run_command(f"cat {result_file}",roles=h)
cat_result


PLAY [all] ************************************************************************************************************************************************

TASK [cat /root/.ansible_async/882274803934.27727] ********************************************************************************************************
 [started TASK: cat /root/.ansible_async/882274803934.27727 on parasilo-12.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]


CommandResult(host='parasilo-12.rennes.grid5000.fr', task='cat /root/.ansible_async/882274803934.27727', status='OK', payload={'cmd': 'cat /root/.ansible_async/882274803934.27727', 'stdout': '{"cmd": "for i in $(seq 1 10); do sleep 1; echo toto; done", "stdout": "toto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto", "stderr": "", "rc": 0, "start": "2021-08-27 17:00:37.397249", "end": "2021-08-27 17:00:47.407273", "delta": "0:00:10.010024", "changed": true, "invocation": {"module_args": {"_raw_params": "for i in $(seq 1 10); do sleep 1; echo toto; done", "_uses_shell": true, "warn": true, "stdin_add_newline": true, "strip_empty_ends": true, "argv": null, "chdir": null, "executable": null, "creates": null, "removes": null, "stdin": null}}}', 'stderr': '', 'rc': 0, 'start': '2021-08-27 17:00:58.145872', 'end': '2021-08-27 17:00:58.148476', 'delta': '0:00:00.002604', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'cat /root/.ansible_async/882274803934.27727', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['{"cmd": "for i in $(seq 1 10); do sleep 1; echo toto; done", "stdout": "toto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto", "stderr": "", "rc": 0, "start": "2021-08-27 17:00:37.397249", "end": "2021-08-27 17:00:47.407273", "delta": "0:00:10.010024", "changed": true, "invocation": {"module_args": {"_raw_params": "for i in $(seq 1 10); do sleep 1; echo toto; done", "_uses_shell": true, "warn": true, "stdin_add_newline": true, "strip_empty_ends": true, "argv": null, "chdir": null, "executable": null, "creates": null, "removes": null, "stdin": null}}}'], 'stderr_lines': [], '_ansible_no_log': False})

In [30]:
# the result_file content is json encoded so decoding it
import json
print(json.loads(cat_result[0].stdout)["stdout"])

toto
toto
toto
toto
toto
toto
toto
toto
toto
toto


## Using variables

### Same variable value for everyone

Nothing surprising here, you can use regular python interpolation (e.g a `f-string`).
String are interpolated by the interpreter before being manipulated.

In [31]:
host_to_ping = roles["control"][0].alias
host_to_ping

results = en.run_command(f"ping -c 5 {host_to_ping}", roles=roles)
results


PLAY [all] ************************************************************************************************************************************************

TASK [ping -c 5 parasilo-12.rennes.grid5000.fr] ***********************************************************************************************************
 [started TASK: ping -c 5 parasilo-12.rennes.grid5000.fr on parasilo-12.rennes.grid5000.fr]
 [started TASK: ping -c 5 parasilo-12.rennes.grid5000.fr on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]


CommandResult(host='parasilo-12.rennes.grid5000.fr', task='ping -c 5 parasilo-12.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-12.rennes.grid5000.fr', 'stdout': 'PING parasilo-12.rennes.grid5000.fr (172.16.97.12) 56(84) bytes of data.\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=1 ttl=64 time=0.023 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=2 ttl=64 time=0.017 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=3 ttl=64 time=0.014 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=4 ttl=64 time=0.028 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=5 ttl=64 time=0.015 ms\n\n--- parasilo-12.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 99ms\nrtt min/avg/max/mdev = 0.014/0.019/0.028/0.006 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-27 17:01:02.313595', 'end': '2021-08-27 17:01:06.418418', 'delta': '0:00:04.104823', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-12.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-12.rennes.grid5000.fr (172.16.97.12) 56(84) bytes of data.', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=1 ttl=64 time=0.023 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=2 ttl=64 time=0.017 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=3 ttl=64 time=0.014 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=4 ttl=64 time=0.028 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=5 ttl=64 time=0.015 ms', '', '--- parasilo-12.rennes.grid5000.fr ping statistics ---', '5 packets transmitted, 5 received, 0% packet loss, time 99ms', 'rtt min/avg/max/mdev = 0.014/0.019/0.028/0.006 ms'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-20.rennes.grid5000.fr', task='ping -c 5 parasilo-12.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-12.rennes.grid5000.fr', 'stdout': 'PING parasilo-12.rennes.grid5000.fr (172.16.97.12) 56(84) bytes of data.\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=1 ttl=64 time=0.156 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=2 ttl=64 time=0.186 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=3 ttl=64 time=0.186 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=4 ttl=64 time=0.148 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=5 ttl=64 time=0.098 ms\n\n--- parasilo-12.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 77ms\nrtt min/avg/max/mdev = 0.098/0.154/0.186/0.035 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-27 17:01:02.340911', 'end': '2021-08-27 17:01:06.424846', 'delta': '0:00:04.083935', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-12.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-12.rennes.grid5000.fr (172.16.97.12) 56(84) bytes of data.', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=1 ttl=64 time=0.156 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=2 ttl=64 time=0.186 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=3 ttl=64 time=0.186 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=4 ttl=64 time=0.148 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=5 ttl=

In [32]:
[(r.host, r.stdout) for r in results]

[('parasilo-12.rennes.grid5000.fr',
  'PING parasilo-12.rennes.grid5000.fr (172.16.97.12) 56(84) bytes of data.\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=1 ttl=64 time=0.023 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=2 ttl=64 time=0.017 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=3 ttl=64 time=0.014 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=4 ttl=64 time=0.028 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=5 ttl=64 time=0.015 ms\n\n--- parasilo-12.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 99ms\nrtt min/avg/max/mdev = 0.014/0.019/0.028/0.006 ms'),
 ('parasilo-20.rennes.grid5000.fr',
  'PING parasilo-12.rennes.grid5000.fr (172.16.97.12) 56(84) bytes of data.\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=1 ttl=64 time=0.156 ms\n64 bytes from parasilo-12.rennes.grid

### Using templates / Ansible variables

There's an alternative way to pass a variable to a task: using `extra_vars`.
The difference with the previous case (python interpreted variables) is the fact that the variable is interpolated right before execution happens on the remote node.
One could imagine the the value is broadcasted to all nodes and replaced right before the execution.

To indicate that we want to use this kind of variables, we need to pass its value using the `extra_vars` dictionnary and use a template (`{{ ... }}`) in the task description.

In [33]:
host_to_ping = roles["control"][0].alias
host_to_ping

results = en.run_command("ping -c 5 {{ my_template_variable }}", roles=roles, extra_vars=dict(my_template_variable=host_to_ping))
results


PLAY [all] ************************************************************************************************************************************************

TASK [ping -c 5 parasilo-12.rennes.grid5000.fr] ***********************************************************************************************************
 [started TASK: ping -c 5 {{ my_template_variable }} on parasilo-12.rennes.grid5000.fr]
 [started TASK: ping -c 5 {{ my_template_variable }} on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]


CommandResult(host='parasilo-12.rennes.grid5000.fr', task='ping -c 5 parasilo-12.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-12.rennes.grid5000.fr', 'stdout': 'PING parasilo-12.rennes.grid5000.fr (172.16.97.12) 56(84) bytes of data.\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=1 ttl=64 time=0.022 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=2 ttl=64 time=0.031 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=3 ttl=64 time=0.030 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=4 ttl=64 time=0.031 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=5 ttl=64 time=0.031 ms\n\n--- parasilo-12.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 92ms\nrtt min/avg/max/mdev = 0.022/0.029/0.031/0.003 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-27 17:01:06.838372', 'end': '2021-08-27 17:01:10.930468', 'delta': '0:00:04.092096', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-12.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-12.rennes.grid5000.fr (172.16.97.12) 56(84) bytes of data.', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=1 ttl=64 time=0.022 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=2 ttl=64 time=0.031 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=3 ttl=64 time=0.030 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=4 ttl=64 time=0.031 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=5 ttl=64 time=0.031 ms', '', '--- parasilo-12.rennes.grid5000.fr ping statistics ---', '5 packets transmitted, 5 received, 0% packet loss, time 92ms', 'rtt min/avg/max/mdev = 0.022/0.029/0.031/0.003 ms'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-20.rennes.grid5000.fr', task='ping -c 5 parasilo-12.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-12.rennes.grid5000.fr', 'stdout': 'PING parasilo-12.rennes.grid5000.fr (172.16.97.12) 56(84) bytes of data.\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=1 ttl=64 time=0.158 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=2 ttl=64 time=0.165 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=3 ttl=64 time=0.189 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=4 ttl=64 time=0.188 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=5 ttl=64 time=0.131 ms\n\n--- parasilo-12.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 90ms\nrtt min/avg/max/mdev = 0.131/0.166/0.189/0.022 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-27 17:01:06.846123', 'end': '2021-08-27 17:01:10.937071', 'delta': '0:00:04.090948', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-12.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-12.rennes.grid5000.fr (172.16.97.12) 56(84) bytes of data.', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=1 ttl=64 time=0.158 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=2 ttl=64 time=0.165 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=3 ttl=64 time=0.189 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=4 ttl=64 time=0.188 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=5 ttl=

### Host specific variables

In the above, we've seen how a common value can be broadcasted to all remote nodes.  What if we want host specific value ?

For instance in our case we'd like `host 1` to ping `host 2` and `host 2` to ping `host 1`. That make the `host_to_ping` variable host-specific.

For this purpose you can use the `extra` attribute of the `Host` objects and use a template as before.

In [34]:
control_host = roles["control"][0]
compute_host = roles["compute"][0]
control_host.extra.update(host_to_ping=compute_host.address)
compute_host.extra.update(host_to_ping=control_host.address)
control_host

ip
::1/128
127.0.0.1/8
ip
172.16.97.12/20
fe80::eef4:bbff:fed0:f148/64


> Note that the `extra` attribute is mutable :(

In [35]:
results = en.run_command("ping -c 5 {{ host_to_ping }}", roles=roles)
results


PLAY [all] ************************************************************************************************************************************************

TASK [ping -c 5 parasilo-20.rennes.grid5000.fr] ***********************************************************************************************************
 [started TASK: ping -c 5 {{ host_to_ping }} on parasilo-12.rennes.grid5000.fr]
 [started TASK: ping -c 5 {{ host_to_ping }} on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-12.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]


CommandResult(host='parasilo-12.rennes.grid5000.fr', task='ping -c 5 parasilo-20.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-20.rennes.grid5000.fr', 'stdout': 'PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.131 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.143 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.183 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.190 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl=64 time=0.208 ms\n\n--- parasilo-20.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 82ms\nrtt min/avg/max/mdev = 0.131/0.171/0.208/0.029 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-27 17:01:11.391793', 'end': '2021-08-27 17:01:15.474605', 'delta': '0:00:04.082812', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-20.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.131 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.143 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.183 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.190 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl=64 time=0.208 ms', '', '--- parasilo-20.rennes.grid5000.fr ping statistics ---', '5 packets transmitted, 5 received, 0% packet loss, time 82ms', 'rtt min/avg/max/mdev = 0.131/0.171/0.208/0.029 ms'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-20.rennes.grid5000.fr', task='ping -c 5 parasilo-12.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-12.rennes.grid5000.fr', 'stdout': 'PING parasilo-12.rennes.grid5000.fr (172.16.97.12) 56(84) bytes of data.\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=1 ttl=64 time=0.131 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=2 ttl=64 time=0.156 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=3 ttl=64 time=0.162 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=4 ttl=64 time=0.160 ms\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=5 ttl=64 time=0.195 ms\n\n--- parasilo-12.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 98ms\nrtt min/avg/max/mdev = 0.131/0.160/0.195/0.025 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-27 17:01:11.446236', 'end': '2021-08-27 17:01:15.544964', 'delta': '0:00:04.098728', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-12.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-12.rennes.grid5000.fr (172.16.97.12) 56(84) bytes of data.', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=1 ttl=64 time=0.131 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=2 ttl=64 time=0.156 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=3 ttl=64 time=0.162 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=4 ttl=64 time=0.160 ms', '64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=5 ttl=

In [36]:
[(r.host, r.stdout) for r in results]

[('parasilo-12.rennes.grid5000.fr',
  'PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.131 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.143 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.183 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.190 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl=64 time=0.208 ms\n\n--- parasilo-20.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 82ms\nrtt min/avg/max/mdev = 0.131/0.171/0.208/0.029 ms'),
 ('parasilo-20.rennes.grid5000.fr',
  'PING parasilo-12.rennes.grid5000.fr (172.16.97.12) 56(84) bytes of data.\n64 bytes from parasilo-12.rennes.grid5000.fr (172.16.97.12): icmp_seq=1 ttl=64 time=0.131 ms\n64 bytes from parasilo-12.rennes.grid

## Cleaning

In [37]:
provider.destroy()

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading 1816396 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Killing the job (rennes, 1816396)
